In [1]:
#if there are problems with imports run:
#!pip install ipynb
#!pip install import-ipynb
#!pip install ase
#!pip install git+https://github.com/henriasv/lammps-logfile.git
#!pip install scipy
#!pip install PySimpleGUI
#!pip install ipympl

import import_ipynb
from Methods import *

sg.theme("DarkAmber")

importing Jupyter notebook from Methods.ipynb


'DarkAmber'

In [2]:
combox=[]
comboy=[]
combon=[]

window,fit_window, rdf_window = make_main_window(),None, None
window.Maximize()
fit_lines=[]
fit_list=[]
lim=[]
is_lines=[]
iline=None
ipoint = []

while True:
    win, event, values = sg.read_all_windows()
    #print(event, values)
    if event in (sg.WIN_CLOSED, 'Exit'):  # always,  always give a way out!
        if win==fit_window:
            win.close()
            fit_window=None
            fit_list.clear()
            fit_lines.clear()
            lim.clear()
        elif win == rdf_window:
            win.close()
            rdf_window=None
            rdf_data=None
            rdf_dict = {}
        elif win==window:
            win.close()
            if fit_window:
                fit_window.close()
            if rdf_window:
                rdf_window.close()
            break

    elif event == 'Plot':
         plot_main(log, unit_types, window['fig_cv'].TKCanvas, window['controls_cv'].TKCanvas, values)

    elif event == 'Linear Fit' and not fit_window:
        fit_window=make_fit_window()
        fit_window['combo_xfit'].update(value=values['combo_x1'], values=combox)
        fit_window['combo_yfit'].update(value=values['combo_y1'], values=comboy)
        fit_window['combo_nfit'].update(value=values['combo_n1'], values=combon)

    elif event == 'RDF' and not rdf_window:
        rdf_window = make_rdf_window()
        rdf_window['combo_xrdf'].update(value=values['combo_x1'], values=combox)
        rdf_window['combo_yrdf'].update(value=values['combo_y1'], values=comboy)
        rdf_window['combo_nrdf'].update(value=values['combo_n1'], values=combon)
        
    elif event == "file_main":    
        file_names = values["file_main"].split(';')
        for file_name in file_names:
            with open(file_name,'r') as f:
                log,unit_types=read_log_file(f)
                if log.get_keywords() == [] and unit_types == []:
                    sg.popup_ok('An error occurred while attempting to open\n' + file_name + '.' ,title = 'Error')
                else:
                    #window['tg1'].add_tab(sg.Tab('tab2', layout = [[sg.T('tab2t')]]))
                    #window['tab1'].update(visible=False)
                    combox=log.get_keywords()
                    comboy=log.get_keywords()
                    combon=np.arange(1,len(unit_types)+1).tolist()
                    for i in range(1,5):
                        window['combo_n'+str(i)].update(value=combon[0], values=combon)
                        window['combo_x'+str(i)].update(value=combox[0], values=combox)
                        window['combo_y'+str(i)].update(value=comboy[0], values=comboy)
                        
    elif event == "file_rdf":   
        file_names = values["file_rdf"].split(';')
        for file_name in file_names:
            with open('data.rdf','r') as f:
                rdf_dict = read_RDF_file(f)
            
    elif '+CLICKED+' in event:
        if len(is_lines) == 2:
            is_lines[0] = is_lines[1]
            is_lines[1] = event[2][0]
        else:
            is_lines.append(event[2][0])
    
    elif event == "Intersection point":
        if len(is_lines) == 2:
            a1,b1 = fit_list[is_lines[0]][1],fit_list[is_lines[0]][2]
            a2,b2 = fit_list[is_lines[1]][1],fit_list[is_lines[1]][2]
            xi, yi = intersection(a1,b1,a2,b2)
            fit_window['interception'].update(value = str(xi) + ' ' + ux_fit)
            if iline:
                line = iline.pop(0)
                line.remove()
                fit_fig.canvas.draw()
            if xi<x_fit.max() and xi>x_fit.min():            
                iline = ax_fit.plot([xi, xi],[y_fit.min(),y_fit.max()], linestyle = 'dashed', color = 'red')
                fit_fig.canvas.draw()
        
    elif event == 'Plotfit':
        fit_fig, ax_fit, x_fit, y_fit, ux_fit, uy_fit = plot_fit(log, unit_types, fit_window['fig_cv_fit'].TKCanvas,
                                                                 fit_window['controls_cv_fit'].TKCanvas, values)
        lim.clear()
        fit_list.clear()
        fit_lines.clear()
        is_lines.clear()
        fit_window['table'].update(fit_list)
        
        def on_click_fit(event):
            lim.append(event.xdata)  

            if len(lim)==2:
                lim1 = np.where(x_fit == min(x_fit, key=lambda z: abs(lim[0] - z)))[0][0]
                lim2 = np.where(x_fit == min(x_fit, key=lambda z: abs(lim[1] - z)))[0][0]
                a, b, unit = linearFit(x_fit, y_fit, ux_fit, uy_fit, lim1, lim2)

                x_max, x_min = x_fit.max(), x_fit.min()
                x_line = np.arange(x_min, x_max, (x_max - x_min)/1000)
                line=ax_fit.plot(x_line,a*x_line+b)
                fit_fig.canvas.draw()
                fit_lines.append(line)

                fit_list.append([line[0].get_color(), a, b, lim1, lim2])
                fit_window['table'].update(fit_list)
                lim.clear()
        
        cursor =wg.Cursor(ax_fit, useblit=True, color='red',horizOn=False)
        fit_fig.canvas.mpl_connect("button_press_event", on_click_fit)
        fit_fig.canvas.draw()
        
    elif event == 'Plotrdf':
        fig_cv = rdf_window['fig_cv_rdf'].TKCanvas
        tb_cv = rdf_window['controls_cv_rdf'].TKCanvas
        n = values['combo_nrdf'] - 1
        xq = values['combo_xrdf']
        yq = values['combo_yrdf']
        rdf_fig, ax_rdf, ax_rdf2, x_rdf, y_rdf, ux_rdf, uy_rdf = plot_rdf(log, unit_types, fig_cv, tb_cv, xq, yq, n)
        def on_click_rdf(event):
            x0 = event.xdata
            y0 = event.ydata

            if not ipoint == []:
                point = ipoint[0].pop(0)
                point.remove()
                del ipoint[0]
                rdf_fig.canvas.draw()
            
            x, y = find_closest_point(x_rdf, y_rdf, x0, y0)
            if not rdf_dict == {}:
                index = np.where(x_rdf == x)[0][0]
                step = log.get('Step',n)
                key = step[index]
                key_list = list(rdf_dict.keys())
                key_list = [int(x) for x in key_list]
                key_list=np.array(key_list)
                key = find_nearest_value(key_list, key)
                new_index = np.where(step == key)[0]
                rdf_data = np.array(rdf_dict[str(key)])

                x = x_rdf[new_index]
                y = y_rdf[new_index]

                ax_rdf2.cla()
                ax_rdf2.plot(rdf_data[:,1], rdf_data[:,2])
                ax_rdf2.set(xlabel = "Distance [A]", ylabel = "RDF")
                rdf_fig.canvas.draw() 
            
                ipoint.append(ax_rdf.plot(x, y, color = 'red', marker = 'o'))
            
            rdf_fig.canvas.draw() 
        
        cursor =wg.Cursor(ax_rdf, useblit=True, color='red')
        rdf_fig.canvas.mpl_connect("button_press_event", on_click_rdf)
        rdf_fig.canvas.draw()        
        
window.close()